In [1]:
# load packages
import numpy as np 
from scipy.io   import  loadmat
from scipy import stats
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
import statsmodels.api as sm 
from statsmodels.formula.api import ols

#### import and set actual anova params

In [2]:
# Main Directory of processed file from MatLab
#MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AvgCRFs\\' # set dir
MainDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\AmpResponseDiffs\\' # set dir
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Files on hand: {d}')
##############################################
FileN = d[0] # choose one                        
file_path1 = os.path.join(MainDir, FileN) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

Files on hand: ['LTP_ResponseDifferences_20240223_1325.pkl', 'PostPreResponseDifferences_20240220_1029.pkl']
Current WD: C:\plimon\LTP_analysis\RCA_F1\AmpResponseDiffs\LTP_ResponseDifferences_20240223_1325.pkl
Does File #1 Exist? True


In [3]:
loadData = pkl.load(open(file_path1,'rb'))
print(loadData.keys())

dict_keys(['DiffData', 'ErrVals', 'SubNames', 'DictMainKeys', 'ContLevs', 'crfLabs', 'VoiceMemo'])


In [4]:
data = loadData['DiffData']
# wont load error bars just yet
data_labels = loadData['DictMainKeys']
SubNames = np.array(loadData['SubNames'])
note = loadData['VoiceMemo']
contrast_levels = loadData['ContLevs']
contrast_labs = loadData['crfLabs']
print(note)
NumSubs = int(len(SubNames))
print(data_labels) # labels for nr plots

['Cont:Post - pre responses for all conditions for all subs, sub x contrast diff']
['attnL F1', 'attnL F2', 'attnR F1', 'attnR F2']


#### Stack all condition differences for 1 subject - make into a function

In [5]:
NumBins = 6 # number of contrasts
NumHarms = 2 # number of harmonic data: 2F1, 4F1
NumComp = 0 # first component from RCA
NumConds = 4

In [6]:
RowPop = NumBins * NumHarms * NumConds # 6 x 2 x 4
SubRows = NumBins*NumHarms
ContrastInd = np.arange(0,NumBins)
ContrastArr = np.arange(RowPop) % NumBins

In [7]:
AttnXArr = np.zeros((RowPop))
FreqFiltArr = np.zeros((RowPop))
HarmArr = np.zeros((RowPop))
# load subject response differnces across conditions

In [8]:
for co in range(NumConds):
    HarmArr[(co*SubRows):(co+1)*SubRows]  = [0] * NumBins + [1] * NumBins # index harmonic data:  2F / 4F 

for co in range(NumConds):
    if co == 0 or co == 1: # Index if Data is AttnL or AttnR
        AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 0
    else:
        AttnXArr[(co*SubRows):(co+1)*(SubRows)] = 1

for co in range(NumConds):
    if co % 2:          # Index what data was F1 = 6 Hz or F2 = 7.5 Hz 
        FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [1]*SubRows
    else:
        FreqFiltArr[(co*SubRows):(co+1)*SubRows] = [0]*SubRows

In [9]:
def MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi, dictIn):
    SubjArr = [soi] * RowPop # Subject label
    ValArr = np.zeros((RowPop)) # INDEPENDENT VARIABLE 12 x 4  = 48 rows per val
    # load subject response differnces across conditions
    for co in range(NumConds):
        dIn = dictIn[co][soi,:] 
        ValArr[(co*SubRows):(co+1)*SubRows] = dIn # Sort Data Values 
    
    dataOut = {'Subj': SubjArr,
                'AttnX': AttnXArr,
                'FreqFilt': FreqFiltArr,
                'Harmonic': HarmArr,
                'Contrast': ContrastArr,
                'Vals': ValArr}
    df = pd.DataFrame(dataOut)
    return df

In [10]:
Sub_df = {}
for su in range(NumSubs):
    Sub_df[su]= MakeDataFramePerSubject(RowPop,SubRows,ContrastArr,AttnXArr,FreqFiltArr,HarmArr,NumConds,soi=su,dictIn=data)

In [11]:
Sub_df[21]

,Subj,AttnX,FreqFilt,Harmonic,Contrast,Vals
0,21,0.0,0.0,0.0,0,-0.412005
1,21,0.0,0.0,0.0,1,-1.008272
2,21,0.0,0.0,0.0,2,-1.257229
3,21,0.0,0.0,0.0,3,-0.751122
4,21,0.0,0.0,0.0,4,-0.516568
5,21,0.0,0.0,0.0,5,-0.486600
6,21,0.0,0.0,1.0,0,-0.690438
7,21,0.0,0.0,1.0,1,-0.561833
8,21,0.0,0.0,1.0,2,-0.263983
9,21,0.0,0.0,1.0,3,-0.213367


In [12]:
anova_array = np.zeros((RowPop*NumSubs,6))
print(anova_array.shape)

headerArr =list(Sub_df[0].columns.values)
print(headerArr)

(1200, 6)
['Subj', 'AttnX', 'FreqFilt', 'Harmonic', 'Contrast', 'Vals']


In [13]:
for su in range(NumSubs):
    aIn = np.array(Sub_df[su])
    anova_array[(RowPop*su):(RowPop*(su+1)),:] = aIn

In [14]:
anova_df = pd.DataFrame(anova_array, columns = headerArr)

In [15]:
anova_df

,Subj,AttnX,FreqFilt,Harmonic,Contrast,Vals
0,0.0,0.0,0.0,0.0,0.0,-0.053774
1,0.0,0.0,0.0,0.0,1.0,-0.395514
2,0.0,0.0,0.0,0.0,2.0,0.293846
3,0.0,0.0,0.0,0.0,3.0,0.278024
4,0.0,0.0,0.0,0.0,4.0,-0.058382
...,...,...,...,...,...,...
1195,24.0,1.0,1.0,1.0,1.0,-0.210037
1196,24.0,1.0,1.0,1.0,2.0,0.035446
1197,24.0,1.0,1.0,1.0,3.0,0.010592
1198,24.0,1.0,1.0,1.0,4.0,0.058085


#### Run ANOVA

In [16]:
#perform two-way ANOVA
model = ols('Vals ~ C(AttnX) + C(FreqFilt) + C(Harmonic)+ C(Contrast) + C(Harmonic):C(FreqFilt)', data=anova_df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(AttnX),0.756362,1.0,1.568224,0.210711
C(FreqFilt),3.161150,1.0,6.554259,0.010586
C(Harmonic),0.036925,1.0,0.076559,0.782064
C(Contrast),2.016581,5.0,0.836227,0.523934
C(Harmonic):C(FreqFilt),0.136154,1.0,0.282300,0.595297
Residual,573.942696,1190.0,NaN,NaN
